In [14]:
from datasets import load_dataset
import ollama
import pandas as pd

In [11]:
from datasets import load_dataset

# other options include: electronics, grocery, movies, restaurants, tripadvisor
dataset = load_dataset("subjqa", "books")

Generating train split:   0%|          | 0/1314 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/345 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/256 [00:00<?, ? examples/s]

In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
        num_rows: 1314
    })
    test: Dataset({
        features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
        num_rows: 345
    })
    validation: Dataset({
        features: ['domain', 'nn_mod', 'nn_asp', 'query_mod', 'query_asp', 'q_reviews_id', 'question_subj_level', 'ques_subj_score', 'is_ques_subjective', 'review_id', 'id', 'title', 'context', 'question', 'answers'],
        num_rows: 256
    })
})

In [21]:
df = pd.DataFrame(dataset['validation']).sample(n=10)

In [22]:
df

,domain,nn_mod,nn_asp,query_mod,query_asp,q_reviews_id,question_subj_level,ques_subj_score,is_ques_subjective,review_id,id,title,context,question,answers
199,books,simplistic,book,positive,effect,558841810b1a5d53af30d1fef9507be1,1,0.000,False,fd84cc64bf2ddd9287e6d7e9468c0988,8dd14a14212888df773d5266ac59f7dc,074356619X,"Disappointing, especially if you are wanting t...",How is it the effect?,"{'text': [], 'answer_start': [], 'answer_subj_..."
80,books,layered,story,epic,story,45ab90bddca5aafa6d0bd6bc58246303,1,0.000,False,a35580f8f680ac71a532689c2a0213b3,c9eb09eb2677a17a74f0a9e87fcee9d8,0307744434,I have to admit that I went into reading The N...,What is the history?,"{'text': [], 'answer_start': [], 'answer_subj_..."
209,books,intriguing,story,intriguing,story line,cbd0ec7df05572f88ff60b50e26fe135,1,0.000,False,4b29c3df5d4efb6006b79d41b787871e,1d23f9d4444d4c27cc880c657ec23542,0857521012,Story is intriguing though incredulous. But I ...,How is the story line?,{'text': ['Story is intriguing though incredul...
177,books,compelling,idea,wrong,thing,3dced0c869cdd7be37c484e0a8bb8b02,1,0.375,False,77197c684d8c398d43018aa34a8b0444,966e74318499855aa831e5eb49271065,0439023513,The idea of The Hunger Games is disturbing yet...,Is there any other thing ?,"{'text': [], 'answer_start': [], 'answer_subj_..."
11,books,meaningful,book,fictional,story,5c4bd5cef05b5d65607133a668dd5c7b,1,0.000,False,65daf5a23342ba0942909ddb79118891,43a89c6bbd98e1d815b5c7df29c51f83,0061950726,I have a friend who was actually an orphan and...,What is the story like?,"{'text': ['very good'], 'answer_start': [140],..."
102,books,evident,passion,not worth to read,book,63fde6fa45d715fa40afb86b3c6d34c8,1,0.600,True,3d6147840f59afdb65d437842d3e4615,55521af462c72a3f76700a729c6cb383,0316055433,"A hauntingly beautiful character study, brilli...",How good is the book?,"{'text': [], 'answer_start': [], 'answer_subj_..."
45,books,worthy,book,captivating,writing style,ce5613fb4914feda72e38f05530798c6,3,0.000,False,30cdc1c149a3f8c647097e6156f11aa4,dbcd1c6e564aa51e6f77d416f0b7e075,0151008116,I thought this book was incredible and definat...,How is writing style?,{'text': ['yes it seems a bit pointless at tim...
223,books,descriptive,author,great,character development,3e401d77458a061ccb4236e3124f0527,1,0.750,True,015f58709eab5523954e1987ba669699,7dce6b53f70d6ac062b3c7c95d6fbdc4,1469984202,I had never considered myself much of a fan of...,Is character development great?,"{'text': [], 'answer_start': [], 'answer_subj_..."
89,books,more complex,plot,satisfying,book,a856dd78d1dd5ffc4516bd69beb61991,2,0.000,False,2536de019bbfb98ca00f34c705d82e29,0aac7354335e17d2a1ac126bec1819f3,0312655479,I love this book! I was fortunate to receive a...,What is your sentiment about this book?,"{'text': ['I love this book!'], 'answer_start'..."
16,books,fun,book,fun,read,1c47f65bd1cc09dc0143a9b1a3bb0adc,1,0.000,False,d142ed1f1b863571cf71a26dad7ee002,43bcbb45665623ff0084836610deee50,0062059939,This ARC was provided by Edelweiss in exchange...,How is the read?,"{'text': [], 'answer_start': [], 'answer_subj_..."


In [26]:
def getModelFeedback(row, model):
    context = row["context"]
    question = row["question"]
    answer = row["answers"]
    
    prompt = f"I am providing you the context, question and answer. Please tell me if the answer text is correct or not.Just provide 'correct' or 'wrong'. Do not provide any explaination or anything else.\nContext: {context}\nQuestion: {question}\nAnswer: {answer}"
    
    response = ollama.chat(model=model, messages=[{
        'role': 'user',
        'content': prompt,
    }])
    
    return response['message']['content']
    

In [27]:
df["Mistral Verdict"] = df.apply(getModelFeedback, model="mistral", axis=1)

In [28]:
df

,domain,nn_mod,nn_asp,query_mod,query_asp,q_reviews_id,question_subj_level,ques_subj_score,is_ques_subjective,review_id,id,title,context,question,answers,Mistral Verdict
199,books,simplistic,book,positive,effect,558841810b1a5d53af30d1fef9507be1,1,0.000,False,fd84cc64bf2ddd9287e6d7e9468c0988,8dd14a14212888df773d5266ac59f7dc,074356619X,"Disappointing, especially if you are wanting t...",How is it the effect?,"{'text': [], 'answer_start': [], 'answer_subj_...",Wrong. The question should have a clear subje...
80,books,layered,story,epic,story,45ab90bddca5aafa6d0bd6bc58246303,1,0.000,False,a35580f8f680ac71a532689c2a0213b3,c9eb09eb2677a17a74f0a9e87fcee9d8,0307744434,I have to admit that I went into reading The N...,What is the history?,"{'text': [], 'answer_start': [], 'answer_subj_...",Wrong. The context does not provide any infor...
209,books,intriguing,story,intriguing,story line,cbd0ec7df05572f88ff60b50e26fe135,1,0.000,False,4b29c3df5d4efb6006b79d41b787871e,1d23f9d4444d4c27cc880c657ec23542,0857521012,Story is intriguing though incredulous. But I ...,How is the story line?,{'text': ['Story is intriguing though incredul...,correct.\nAnswer text matches the given conte...
177,books,compelling,idea,wrong,thing,3dced0c869cdd7be37c484e0a8bb8b02,1,0.375,False,77197c684d8c398d43018aa34a8b0444,966e74318499855aa831e5eb49271065,0439023513,The idea of The Hunger Games is disturbing yet...,Is there any other thing ?,"{'text': [], 'answer_start': [], 'answer_subj_...",Wrong. The question seems to be asking if the...
11,books,meaningful,book,fictional,story,5c4bd5cef05b5d65607133a668dd5c7b,1,0.000,False,65daf5a23342ba0942909ddb79118891,43a89c6bbd98e1d815b5c7df29c51f83,0061950726,I have a friend who was actually an orphan and...,What is the story like?,"{'text': ['very good'], 'answer_start': [140],...",correct.
102,books,evident,passion,not worth to read,book,63fde6fa45d715fa40afb86b3c6d34c8,1,0.600,True,3d6147840f59afdb65d437842d3e4615,55521af462c72a3f76700a729c6cb383,0316055433,"A hauntingly beautiful character study, brilli...",How good is the book?,"{'text': [], 'answer_start': [], 'answer_subj_...",The answer text is not provided in the contex...
45,books,worthy,book,captivating,writing style,ce5613fb4914feda72e38f05530798c6,3,0.000,False,30cdc1c149a3f8c647097e6156f11aa4,dbcd1c6e564aa51e6f77d416f0b7e075,0151008116,I thought this book was incredible and definat...,How is writing style?,{'text': ['yes it seems a bit pointless at tim...,wrong. The context does mention the writing s...
223,books,descriptive,author,great,character development,3e401d77458a061ccb4236e3124f0527,1,0.750,True,015f58709eab5523954e1987ba669699,7dce6b53f70d6ac062b3c7c95d6fbdc4,1469984202,I had never considered myself much of a fan of...,Is character development great?,"{'text': [], 'answer_start': [], 'answer_subj_...",The answer text is not applicable to the ques...
89,books,more complex,plot,satisfying,book,a856dd78d1dd5ffc4516bd69beb61991,2,0.000,False,2536de019bbfb98ca00f34c705d82e29,0aac7354335e17d2a1ac126bec1819f3,0312655479,I love this book! I was fortunate to receive a...,What is your sentiment about this book?,"{'text': ['I love this book!'], 'answer_start'...",correct.
16,books,fun,book,fun,read,1c47f65bd1cc09dc0143a9b1a3bb0adc,1,0.000,False,d142ed1f1b863571cf71a26dad7ee002,43bcbb45665623ff0084836610deee50,0062059939,This ARC was provided by Edelweiss in exchange...,How is the read?,"{'text': [], 'answer_start': [], 'answer_subj_...",wrong. The answer should provide a rating or ...
